In [ ]:
import seaborn as sns
import pandas as pd

In [ ]:
data = pd.read_csv(
    "../train.csv",
    sep=";",
    dtype={"DATE": str, "CUSTOMER": str, "QUANTITY": int},
    index_col="DATE",
    parse_dates=["DATE"],
)
data

In [ ]:
sns.relplot(
    data,
    x="DATE",
    y="QUANTITY",
    hue="CUSTOMER",
    col="CUSTOMER",
    kind="line",
    aspect=5,
    col_wrap=1,
    facet_kws={"sharey": False},
)

In [ ]:
data["gap"] = data.index.sort_values().diff() > pd.to_timedelta("1 day")
data[data.gap]

In [ ]:
sns.barplot(data, x="DATE", y="gap", hue="CUSTOMER")

In [ ]:
from darts import TimeSeries

series = TimeSeries.from_group_dataframe(
    data,
    group_cols="CUSTOMER",
    value_cols="QUANTITY",
    fill_missing_dates=True,
    freq="D",
)

In [ ]:
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller

transformer_filler = MissingValuesFiller()
series = transformer_filler.transform(series)

In [ ]:
from darts.utils.statistics import check_seasonality

for s in series:
    print(s.static_covariates_values(), check_seasonality(s, 365, max_lag=730))

In [ ]:
from darts.utils.statistics import plot_acf
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(16, 16))

for s, a in zip(series, axs.flat):
    plot_acf(s, 365, max_lag=730, axis=a)
    a.set_title(s.static_covariates_values())

In [ ]:
from darts.utils.statistics import plot_hist
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(16, 16))

for s, a in zip(series, axs.flat):
    plot_hist(s, bins=100, density=False, title=s.static_covariates_values(), ax=a)